In [ ]:
from demo_source import *

In [ ]:
# Functional widget: Text Box
op_textbox = widgets.Text(
    value = list(Demo.edsl_snippets.values())[0], 
    placeholder = list(Demo.edsl_snippets.values())[0], 
    disabled = False
)

# Functional widget: Dropdown
op_dropdown = widgets.Dropdown(
    options = list(Demo.edsl_snippets.keys()),
    value = list(Demo.edsl_snippets.keys())[0],
    disabled = False
)

def dropdown_handler(change):
    op_textbox.value = Demo.edsl_snippets[change['new']]
    op_textbox.placeholder = Demo.edsl_snippets[change['new']]

op_dropdown.observe(dropdown_handler, names='value')

In [ ]:
# Interactive user interface (visible on the left side of the display)
left = widgets.VBox([
    widgets.HTML(value = "<h2>Code</h2>"),
    widgets.HTML(value = "First, select a pre-written EDSL snippet from the list below, or write your own custom operation."),
    widgets.HTML(value = "Then, press Run to see the EDSL code in action!"),
    widgets.HBox([
        widgets.Label(value="Pick an operation:"),
        op_dropdown
    ]),
    widgets.HTML(value = "<code> \
                 def edsl_program(X, Y): <br>\
                 </code>"),
    widgets.VBox(
        [
            widgets.HTML(value = "<code> \
            I, J, K = TensorDims(3) <br>\
            i, j, k = TensorIndexes(3) <br>\
            X.bind_dims(I, K) <br>\
            Y.bind_dims(K, J) <br>\
            R = TensorOutput(I, J) <br>\
            </code>"),
            op_textbox,
            widgets.HTML(value="<code> \
            return R <br>\
            </code>")
        ], 
        layout = widgets.Layout(margin='0 0 0 5%')
    ),
    widgets.Button(
        description='Run',
        disabled=False,
        button_style='success',
        tooltip='Compiles and executes your EDSL program',
        icon='check'
    )
])

In [ ]:
# Live action animation (visible on the right side of the display)
right = widgets.VBox([
    widgets.HTML(value="<h2>Inputs</h2>"),
    widgets.HBox([
        DrawingWidget(Demo.input_a_anim),
        DrawingWidget(Demo.input_b_anim)
    ]),
    widgets.HTML(value="<h2>Result</h2>"),
    DrawingWidget(Demo.output_anim)
])

In [ ]:
# Full-screen layout

hbox_layout = widgets.Layout()
hbox_layout.width = '100%'
hbox_layout.justify_content = 'space-around'

# do this in a flexbox-friendlier way
edsl_title = widgets.HTML(value='<div style="text-align:center"><h1>EDSL Demo</h1></div>')

edsl_subdemo = widgets.HBox([left, right])
edsl_subdemo.layout = hbox_layout

edsl_demo = widgets.VBox([
    edsl_title,
    edsl_subdemo
])

In [ ]:
edsl_demo